In [2]:
#codigo utilizado para obtener los coeficientes de Lyapunov. Elaborado por Gustavo Ardila y Cristian Tibambre
import numpy as np
from scipy.integrate import solve_ivp

#ODE sistema de ecuaciones 
def func(t, v, k, r, b):
    x, y, z, w = v #define todas las variables
    return [ -k * (np.sin(y) - np.sin(w))*np.cos(y) - r*b*np.sin(y), x/(b),  k * (np.sin(y) - np.sin(w))*np.cos(y) - r*b*np.sin(w), z/b ]
def JM(v, sigma, r, b): #define la matriz jacobiana
    x, y, z, w = [k for k in v]
    return np.array([[0, -k*((np.cos(y)**2-(np.sin(y))**2) + np.sin(w)*np.sin(y) - r*b*np.cos(y)), 0, k*np.cos(w)*np.cos(y)], [1/b, 0, 0, 0], [0, k*np.cos(y)*np.cos(w), 0, -k*(np.sin(y)*np.sin(w) + ((np.cos(w))**2-(np.sin(w))**2))-r*b*np.cos(w)], [-b, 0, 0, 1]])
#parametros iniciales
k = 0.5 #constante del resorte
r = 5 #masas 
b = 1 #longitud
U = np.eye(4) #unidades de la funcion nueva
v0 = np.ones(4) #condiciones iniciales.
lyap = [] #array vacio para guardar exponentes


iters=10**3
dt=0.1
tf=iters * dt
#soluciona las ecuaciones diferenciales
sol = solve_ivp(func, [0, tf], v0, t_eval=np.linspace(0, tf, iters), args=(k, r, b))
v_n = sol.y.T #transpone la solucion

#ciclo para iterar desde 0.1 hasta donde se desee 
for k in range(0, iters):
    v0 = v_n[k] #nuevas condiciones iniciales
    U_n = np.matmul(np.eye(4) + JM(v0, k, r, b) * dt, U)
        #hace uso del metodo Gram-Schmidt Orthogonalisation (GSO)
    Q, R = np.linalg.qr(U_n)
    lyap.append(np.log(abs(R.diagonal())))
    
#retorna 4 exponentes de lyapunov
[sum([lyap[k][j] for k in range(iters)]) / (dt * iters) for j in range(4)]

[0.09901313648089884,
 31.810062345338384,
 -4.853826798775816,
 -5.517984098329752]